<a href="https://colab.research.google.com/github/LeilaMemoi/Deep_learning/blob/main/Deep_Learning_Project_Leila_Maritim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

>[DATA](#folderId=1Op6xQrUVNJeBqCu5L-qFKjH5IiLQbb3O&updateTitle=true&scrollTo=qwI6OYWr_Eg0)

>>[Dataset loading](#folderId=1Op6xQrUVNJeBqCu5L-qFKjH5IiLQbb3O&updateTitle=true&scrollTo=Ow1GTx1Y-Kq0)

>>[Dataset characteristics](#folderId=1Op6xQrUVNJeBqCu5L-qFKjH5IiLQbb3O&updateTitle=true&scrollTo=aKuckj75-5kn)

>[DEEP LEARNING MODELS](#folderId=1Op6xQrUVNJeBqCu5L-qFKjH5IiLQbb3O&updateTitle=true&scrollTo=KNeKDJuy_uQu)

>>[Multi-layer perceptron](#folderId=1Op6xQrUVNJeBqCu5L-qFKjH5IiLQbb3O&updateTitle=true&scrollTo=UdOMv3WQAD9o)

>>[1-D Convolutional neural network](#folderId=1Op6xQrUVNJeBqCu5L-qFKjH5IiLQbb3O&updateTitle=true&scrollTo=TBBG35E9ANtu)

>>[Recurrent neural networks](#folderId=1Op6xQrUVNJeBqCu5L-qFKjH5IiLQbb3O&updateTitle=true&scrollTo=cf2fxeSFAX42)

>[CONCLUSION](#folderId=1Op6xQrUVNJeBqCu5L-qFKjH5IiLQbb3O&updateTitle=true&scrollTo=UpCJe8f0BUO_)

>[REFERENCES](#folderId=1Op6xQrUVNJeBqCu5L-qFKjH5IiLQbb3O&updateTitle=true&scrollTo=BERD7oA3ByWT)



# DATA

## 1. Dataset loading

Write a customized dataset class to read and pre-process the data






In [1]:
import numpy as np

In [ ]:

data = np.load(os.path.join(data_CDE_dir, "data_AT1.npz"), allow_pickle=True)
series = data[’series’] # time series (m,l,d) where m is the number of time series in a given domain,
## l is the length of the time series, and d is the number of channels in the image
labels = data[’labels’] # label associated with each time series
dates = data[’dates’] # acquisition dates (datetime package)
class_names = data[’class_names’] # names of the class

## 2. Dataset characteristics

*   What is the learning task?
*   Number of features in the dataset and their types
*   descriptive statistics for quantitative explanatory variables
*   normalize data....if necessary and why
*   class distribution
*   choose cost function to train the deep learning models based on your analysis

# DEEP LEARNING MODELS

That take into account the temporal structure of data, justify their architectures and expalin how you tuned the values for

## 1. Multi-layer perceptron

Architecture

Hyperparameter tuning

Performance

## 2. 1-D Convolutional neural network

Architecture

Hyperparamater tuning

Performance

## 3. Recurrent neural networks

Architecture

Hyperparameter tuning

Performance

# 4. CONCLUSION

Comapre the model performance...expalin how you evaluated the performance to ensure a fair comparison

# 5. REFERENCES